In [ ]:
import os
import uuid
from textwrap import dedent

import shapely
from gait import MAO, Agent, ObserverLoguru, Scratchpad
from rich.pretty import pprint

In [ ]:
model = f"azure/{os.environ['AZURE_API_DEPLOYMENT']}"

In [ ]:
def buffer_geometry(
    geom_uuid: str,
    distance_in_meters: float,
    scratchpad: Scratchpad,
) -> str:
    """Buffer a geometry referenced by a UUID by a distance in meters.
    Return the buffered geometry UUID.

    :param geom_uuid: UUID of a geometry to buffer.
    :param distance_in_meters: The buffer distance in meters.
    :param scratchpad: Scratchpad instance injected at runtime.
    """
    geom = scratchpad[geom_uuid]
    buff = shapely.buffer(geom, distance_in_meters)
    buff_uuid = str(uuid.uuid4())
    scratchpad[buff_uuid] = buff
    return buff_uuid

In [ ]:
buffer_agent = Agent(
    model=model,
    name="BufferAgent",
    description="Buffer a geometry by a distance",
    functions=[buffer_geometry],
    temperature=0.0,
)

In [ ]:
def send_to_buffer_agent():
    """Return a reference to an agent that will buffer a geometry."""
    return buffer_agent

In [ ]:
def wkt_to_geometry(
    wkt: str,
    scratchpad: Scratchpad,
) -> str:
    """Parse WKT string into a geometry UUID. The following are WKT string samples: POINT (30 10) LINESTRING (30 10, 10 30, 40 40) POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))

    :param wkt: The well known text to convert to a geometry UUID.
    :param scratchpad:  Scratchpad instance injected at runtime.
    """
    geom = shapely.from_wkt(wkt, on_invalid="ignore")
    geom_uuid = str(uuid.uuid4())
    scratchpad[geom_uuid] = geom
    return geom_uuid

In [ ]:
wkt_to_geometry_agent = Agent(
    model=model,
    name="WKTAgent",
    description="Convert WKT string to geometry UUID",
    functions=[wkt_to_geometry],
    temperature=0.0,
)

In [ ]:
def send_to_wkt_to_geometry_agent():
    """Return a reference to an agent that will parse WKT string into a geometry UUID."""
    return wkt_to_geometry_agent

In [ ]:
instructions = dedent(
    """
Classify the user request and send it to the correct agent.
If the use request contains a geometry in WKT, make sure to first parse it before performing any operation.

The following are WKT string samples:
    POINT (30 10)
    LINESTRING (30 10, 10 30, 40 40)
    POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))

Make sure to send the request to one agent at the time if they are depending on each other.
"""
).strip()

In [ ]:
classifier = Agent(
    model=model,
    name="Classifier",
    instructions=instructions,
    functions=[
        # send_to_buffer_agent,
        # send_to_wkt_to_geometry_agent,
        buffer_geometry,
        wkt_to_geometry,
    ],
    temperature=0.0,
)

In [ ]:
mao = MAO(
    agent=classifier,
    observer=ObserverLoguru(),
)

In [ ]:
for _ in mao("Buffer by 20 meters POINT (10 30)"):
    if _.content:
        mao.terminate()
        print(_.content)

In [ ]:
for _ in mao.dialog:
    pprint(_, expand_all=True)